In [16]:
import warnings
from datetime import date

import matplotlib.pyplot as plt
import seaborn as sns
import pandas
import numpy as np
from scipy.stats import norm
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

%matplotlib inline
%config InlineBackend.figure_format = "retina"

sns.set(rc={"figure.figsize" : (25, 15)})
sns.set(font_scale=2)
sns.set_style("ticks")

warnings.simplefilter("ignore", FutureWarning)  # ignore some seaborn warnings until they update numpy

In [17]:
df = pandas.read_csv("NHL_API_point_totals_by_team_season_raw_1996-2003.csv")
vegas_df = df[df["team"] == "VGK"].copy()
vegas_df

,team,season_end,total_points,points_rank
673,VGK,2018,109,5
704,VGK,2019,93,15
735,VGK,2020,86,7
766,VGK,2021,82,2
797,VGK,2022,94,17
829,VGK,2023,111,5


In [18]:
vegas_df["season_end"] = vegas_df["season_end"].apply(lambda x : date(month=5, day=1, year=x))
vegas_df["season_end"] = pandas.to_datetime(vegas_df["season_end"])

In [19]:
vegas_df = vegas_df.set_index("season_end")

In [33]:
alphas = np.linspace(0.01, 0.99, 101)

In [42]:
best_alpha = None
lowest_residuals = None
for alpha in alphas:
    ses = SimpleExpSmoothing(vegas_df["total_points"][0:-1]).fit(
        optimized=False, remove_bias=True, smoothing_level=alpha)
    pred = ses.forecast(1)
    residuals = np.abs(vegas_df.iloc[-1]["total_points"] - pred.values[0])
    if lowest_residuals is None or residuals < lowest_residuals:
        best_alpha = alpha
        lowest_residuals = residuals

/Users/diane/miniconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-MAY will be used.
  self._init_dates(dates, freq)
/Users/diane/miniconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-MAY will be used.
  self._init_dates(dates, freq)
/Users/diane/miniconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-MAY will be used.
  self._init_dates(dates, freq)
/Users/diane/miniconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-MAY will be used.
  self._init_dates(dates, freq)
/Users/diane/miniconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency i

In [43]:
best_alpha

0.01

In [44]:
lowest_residuals

18.291229799144872

In [45]:
ses = SimpleExpSmoothing(vegas_df["total_points"]).fit(
    optimized=False, remove_bias=True, smoothing_level=best_alpha)

/Users/diane/miniconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-MAY will be used.
  self._init_dates(dates, freq)


In [46]:
ses.forecast(1)

2024-05-01    95.831236
Freq: AS-MAY, dtype: float64

In [47]:
resid_std = ses.resid.std()

In [48]:
resid_std

11.927451546786228

In [49]:
ses.forecast(1) - (1.96 * resid_std)

2024-05-01    72.453431
Freq: AS-MAY, dtype: float64

In [50]:
ses.forecast(1) + (1.96 * resid_std)

2024-05-01    119.209041
Freq: AS-MAY, dtype: float64